In [34]:
print("Hello")

Hello


In [ ]:
print("Hello")

Hello


In [35]:
import pandas as pd

In [36]:
df = pd.read_csv(r"E:\elevatetrsest\dataset\price_wheat_daily.csv")

In [37]:
df.head()

,date,district_id,district_name,commodity_name,state_name,avg_modal_price,avg_min_price,avg_max_price,calculationType,change
0,2025-03-07,497,Nandurbar,Wheat,Maharashtra,2673.0,2525.5,2716.00,Daily,100.5
1,2025-03-06,497,Nandurbar,Wheat,Maharashtra,2572.5,2470.0,2710.50,Daily,-61.5
2,2025-03-05,497,Nandurbar,Wheat,Maharashtra,2634.0,2500.0,2740.00,Daily,147.5
3,2025-03-04,497,Nandurbar,Wheat,Maharashtra,2486.5,2390.5,2628.75,Daily,-238.5
4,2025-03-03,497,Nandurbar,Wheat,Maharashtra,2725.0,2492.0,2778.00,Daily,-31.5


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30501 entries, 0 to 30500
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             30501 non-null  object 
 1   district_id      30501 non-null  int64  
 2   district_name    30501 non-null  object 
 3   commodity_name   30501 non-null  object 
 4   state_name       30501 non-null  object 
 5   avg_modal_price  30501 non-null  float64
 6   avg_min_price    30501 non-null  float64
 7   avg_max_price    30501 non-null  float64
 8   calculationType  30501 non-null  object 
 9   change           30470 non-null  float64
dtypes: float64(4), int64(1), object(5)
memory usage: 2.3+ MB


In [39]:

df['date_n'] = pd.to_datetime(df['date']).dt.day
df['month'] = pd.to_datetime(df['date']).dt.month
df['year'] = pd.to_datetime(df['date']).dt.year

df.drop('date', axis=1, inplace=True)
df['date'] = df['date_n']
df.drop('date_n', axis=1, inplace=True)

filling null values

In [40]:
 df.fillna('interpolate' , inplace=True)

C:\Users\Shiva\AppData\Local\Temp\ipykernel_541760\2572123431.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'interpolate' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('interpolate' , inplace=True)


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30501 entries, 0 to 30500
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   district_id      30501 non-null  int64  
 1   district_name    30501 non-null  object 
 2   commodity_name   30501 non-null  object 
 3   state_name       30501 non-null  object 
 4   avg_modal_price  30501 non-null  float64
 5   avg_min_price    30501 non-null  float64
 6   avg_max_price    30501 non-null  float64
 7   calculationType  30501 non-null  object 
 8   change           30501 non-null  object 
 9   month            30501 non-null  int32  
 10  year             30501 non-null  int32  
 11  date             30501 non-null  int32  
dtypes: float64(3), int32(3), int64(1), object(5)
memory usage: 2.4+ MB


frequency encoding 

removing unwanted columns

In [42]:
columns = ['district_name' , 'commodity_name' , 'state_name']

for col in columns:
    frequency_encoding = df[col].value_counts()
    df[f'{col}_enc'] = df[col].map(frequency_encoding)

df.drop(columns=columns , axis=1 , inplace=True)
df.drop(columns=['calculationType' , 'district_id' , 'change'] , axis=1 , inplace=True)

In [43]:
df.head()

,avg_modal_price,avg_min_price,avg_max_price,month,year,date,district_name_enc,commodity_name_enc,state_name_enc
0,2673.0,2525.5,2716.00,3,2025,7,981,30501,30501
1,2572.5,2470.0,2710.50,3,2025,6,981,30501,30501
2,2634.0,2500.0,2740.00,3,2025,5,981,30501,30501
3,2486.5,2390.5,2628.75,3,2025,4,981,30501,30501
4,2725.0,2492.0,2778.00,3,2025,3,981,30501,30501


In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30501 entries, 0 to 30500
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   avg_modal_price     30501 non-null  float64
 1   avg_min_price       30501 non-null  float64
 2   avg_max_price       30501 non-null  float64
 3   month               30501 non-null  int32  
 4   year                30501 non-null  int32  
 5   date                30501 non-null  int32  
 6   district_name_enc   30501 non-null  int64  
 7   commodity_name_enc  30501 non-null  int64  
 8   state_name_enc      30501 non-null  int64  
dtypes: float64(3), int32(3), int64(3)
memory usage: 1.7 MB


strategy to tackle the following problem

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import xgboost as xgb
from statsmodels.tsa.statespace.sarimax import SARIMAX

# 1. Prepare Time Series Features
def create_time_features(df):
    """Create additional time-based features"""
    # Convert date columns to datetime if needed
    # Assuming 'date' is day of month, and we need to create a full date column
    df['full_date'] = pd.to_datetime(df[['year', 'month', 'date']].assign(day=lambda x: x['date']))
    
    # Create lag features for each target
    for target in ['avg_modal_price', 'avg_min_price', 'avg_max_price']:
        for lag in [1, 3, 7, 14, 30]:  # Various lag periods
            # Group by relevant features to create lags
            df_grouped = df.groupby(['commodity_name_enc', 'state_name_enc', 'district_name_enc'])
            df[f'{target}_lag_{lag}'] = df_grouped[target].shift(lag)
    
    # Create rolling statistics
    for target in ['avg_modal_price', 'avg_min_price', 'avg_max_price']:
        for window in [7, 14, 30]:
            # Group by relevant features 
            df_grouped = df.groupby(['commodity_name_enc', 'state_name_enc', 'district_name_enc'])
            df[f'{target}_rolling_mean_{window}'] = df_grouped[target].transform(lambda x: x.rolling(window=window, min_periods=1).mean())
            df[f'{target}_rolling_std_{window}'] = df_grouped[target].transform(lambda x: x.rolling(window=window, min_periods=1).std())
    
    # Create cyclical features for month
    df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
    
    # Create day of year feature
    df['day_of_year'] = df['full_date'].dt.dayofyear
    df['day_of_year_sin'] = np.sin(2 * np.pi * df['day_of_year']/365)
    df['day_of_year_cos'] = np.cos(2 * np.pi * df['day_of_year']/365)
    
    # Add trend feature
    df['trend'] = np.arange(len(df))
    
    return df

# 2. Time Series Validation Strategy
def time_series_validation(df, model_func, features, targets, n_splits=5):
    """Validate model using time series cross-validation"""
    tscv = TimeSeriesSplit(n_splits=n_splits)
    results = {target: {'rmse': [], 'mae': [], 'r2': []} for target in targets}
    
    # Sort by date for proper time-based splitting
    df = df.sort_values('full_date')
    
    X = df[features]
    y = df[targets]
    
    for train_idx, test_idx in tscv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        
        # Fill NaN values that might be created by lags
        X_train = X_train.fillna(method='bfill')
        X_test = X_test.fillna(method='bfill')
        
        # Train and evaluate model for each target
        for target in targets:
            model = model_func()
            model.fit(X_train, y_train[target])
            y_pred = model.predict(X_test)
            
            # Calculate metrics
            rmse = np.sqrt(mean_squared_error(y_test[target], y_pred))
            mae = mean_absolute_error(y_test[target], y_pred)
            r2 = r2_score(y_test[target], y_pred)
            
            # Store results
            results[target]['rmse'].append(rmse)
            results[target]['mae'].append(mae)
            results[target]['r2'].append(r2)
    
    # Calculate average metrics
    for target in targets:
        for metric in ['rmse', 'mae', 'r2']:
            results[target][f'avg_{metric}'] = np.mean(results[target][metric])
            
    return results

# 3. XGBoost with Time Series Features
def train_xgboost_time_series():
    """Train XGBoost model with time series considerations"""
    # Prepare data with all features
    processed_df = create_time_features(df)
    
    # Define features and targets
    targets = ['avg_modal_price', 'avg_min_price', 'avg_max_price']
    features = [
        # Original features
        'month', 'year', 'date', 'district_name_enc', 'commodity_name_enc', 'state_name_enc',
        # Time features
        'month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos', 'trend',
        # Lag features (partial list)
        'avg_modal_price_lag_1', 'avg_modal_price_lag_7', 'avg_modal_price_lag_30',
        'avg_min_price_lag_1', 'avg_min_price_lag_7', 'avg_min_price_lag_30',
        'avg_max_price_lag_1', 'avg_max_price_lag_7', 'avg_max_price_lag_30',
        # Rolling statistics (partial list)
        'avg_modal_price_rolling_mean_7', 'avg_modal_price_rolling_std_7',
        'avg_modal_price_rolling_mean_30', 'avg_modal_price_rolling_std_30'
    ]
    
    # Create training and testing datasets with a time-based split
    train_size = int(len(processed_df) * 0.8)
    train_df = processed_df.iloc[:train_size]
    test_df = processed_df.iloc[train_size:]
    
    # Handle missing values that might arise from lag features
    train_df = train_df.fillna(method='bfill')
    test_df = test_df.fillna(method='bfill')
    
    X_train = train_df[features]
    y_train = train_df[targets]
    X_test = test_df[features]
    y_test = test_df[targets]
    
    # Train models for each target
    models = {}
    results = {}
    feature_importances = {}
    
    for target in targets:
        print(f"Training XGBoost model for {target}...")
        
        model = xgb.XGBRegressor(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=6,
            min_child_weight=1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42
        )
        
        model.fit(
            X_train, 
            y_train[target],
            eval_set=[(X_test, y_test[target])],
            early_stopping_rounds=10,
            verbose=False
        )
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate
        rmse = np.sqrt(mean_squared_error(y_test[target], y_pred))
        mae = mean_absolute_error(y_test[target], y_pred)
        r2 = r2_score(y_test[target], y_pred)
        
        print(f"{target} - RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")
        
        # Store model and results
        models[target] = model
        results[target] = {'rmse': rmse, 'mae': mae, 'r2': r2}
        
        # Get feature importance
        feature_importances[target] = pd.DataFrame({
            'Feature': features,
            'Importance': model.feature_importances_
        }).sort_values('Importance', ascending=False)
        
        print("Top 10 important features:")
        print(feature_importances[target].head(10))
        print("\n")
    
    return models, results, feature_importances

# 4. Prophet/SARIMA for Specific Commodity-Location Combinations
def train_sarima_for_specific_combination(commodity_id, state_id, district_id):
    """Train SARIMA model for specific commodity and location"""
    # Filter data for specific combination
    filter_mask = (
        (df['commodity_name_enc'] == commodity_id) & 
        (df['state_name_enc'] == state_id) & 
        (df['district_name_enc'] == district_id)
    )
    
    specific_df = df[filter_mask].copy()
    
    # Create datetime index
    specific_df['date'] = pd.to_datetime(specific_df[['year', 'month', 'date']].assign(day=lambda x: x['date']))
    specific_df = specific_df.set_index('date')
    specific_df = specific_df.sort_index()
    
    # Focus on single target for SARIMA
    target_series = specific_df['avg_modal_price']
    
    # Split data
    train_size = int(len(target_series) * 0.8)
    train_series = target_series[:train_size]
    test_series = target_series[train_size:]
    
    # Fit SARIMA model
    # p, d, q represent AR terms, differencing, and MA terms
    # P, D, Q represent seasonal components
    # s represents seasonal periodicity (12 for monthly, 30 for daily)
    model = SARIMAX(
        train_series,
        order=(1, 1, 1),
        seasonal_order=(1, 1, 1, 30),
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    
    fitted_model = model.fit(disp=False)
    
    # Forecast
    forecast = fitted_model.get_forecast(steps=len(test_series))
    forecast_mean = forecast.predicted_mean
    forecast_ci = forecast.conf_int()
    
    # Evaluate
    rmse = np.sqrt(mean_squared_error(test_series, forecast_mean))
    mae = mean_absolute_error(test_series, forecast_mean)
    
    print(f"SARIMA - RMSE: {rmse:.4f}, MAE: {mae:.4f}")
    
    # Plot results
    plt.figure(figsize=(12, 6))
    plt.plot(train_series.index, train_series, label='Training Data')
    plt.plot(test_series.index, test_series, label='Actual Test Data')
    plt.plot(test_series.index, forecast_mean, label='SARIMA Forecast')
    plt.fill_between(
        test_series.index,
        forecast_ci.iloc[:, 0],
        forecast_ci.iloc[:, 1],
        color='k', alpha=0.1
    )
    plt.title(f'SARIMA Forecast for Commodity {commodity_id} in District {district_id}, State {state_id}')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    return fitted_model, forecast_mean, rmse, mae

# 5. Neural Networks for Sequence Forecasting (Example with TensorFlow)
def build_lstm_model(train_df, test_df, targets, lookback=30):
    """
    Example of using LSTM for time series forecasting
    Note: This requires TensorFlow to be installed
    """
    try:
        import tensorflow as tf
        from tensorflow.keras.models import Sequential
        from tensorflow.keras.layers import LSTM, Dense, Dropout
        from tensorflow.keras.callbacks import EarlyStopping
    except ImportError:
        print("TensorFlow not installed. Skip LSTM implementation.")
        return None, None
    
    # Prepare sequences for LSTM
    def create_sequences(data, target_col, lookback):
        X, y = [], []
        for i in range(len(data) - lookback):
            X.append(data.iloc[i:i+lookback].values)
            y.append(data.iloc[i+lookback][target_col])
        return np.array(X), np.array(y)
    
    # Example for one target
    target = targets[0]  # Using first target for example
    
    # Scale data
    scaler = StandardScaler()
    train_scaled = pd.DataFrame(
        scaler.fit_transform(train_df[['avg_modal_price', 'avg_min_price', 'avg_max_price']]),
        columns=['avg_modal_price', 'avg_min_price', 'avg_max_price']
    )
    test_scaled = pd.DataFrame(
        scaler.transform(test_df[['avg_modal_price', 'avg_min_price', 'avg_max_price']]),
        columns=['avg_modal_price', 'avg_min_price', 'avg_max_price']
    )
    
    # Create sequences
    X_train, y_train = create_sequences(train_scaled, target, lookback)
    X_test, y_test = create_sequences(test_scaled, target, lookback)
    
    # Reshape for LSTM [samples, timesteps, features]
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])
    
    # Build LSTM model
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(lookback, X_train.shape[2])),
        Dropout(0.2),
        LSTM(50),
        Dropout(0.2),
        Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='mse')
    
    # Train
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )
    
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Predict
    y_pred = model.predict(X_test)
    
    # Inverse transform for evaluation
    y_test_original = y_test * scaler.scale_[0] + scaler.mean_[0]
    y_pred_original = y_pred * scaler.scale_[0] + scaler.mean_[0]
    
    # Evaluate
    rmse = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
    mae = mean_absolute_error(y_test_original, y_pred_original)
    
    print(f"LSTM for {target} - RMSE: {rmse:.4f}, MAE: {mae:.4f}")
    
    return model, history

# 6. Ensemble Method
def train_ensemble_model(df, features, targets):
    """
    Train an ensemble of different models and combine their predictions
    """
    # Prepare data
    processed_df = create_time_features(df)
    train_size = int(len(processed_df) * 0.8)
    train_df = processed_df.iloc[:train_size]
    test_df = processed_df.iloc[train_size:]
    
    # Handle missing values
    train_df = train_df.fillna(method='bfill')
    test_df = test_df.fillna(method='bfill')
    
    X_train = train_df[features]
    y_train = train_df[targets]
    X_test = test_df[features]
    y_test = test_df[targets]
    
    # Dictionary to store individual models
    model_predictions = {target: {} for target in targets}
    ensemble_results = {target: {} for target in targets}
    
    for target in targets:
        print(f"Training ensemble for {target}...")
        
        # Model 1: XGBoost
        xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
        xgb_model.fit(X_train, y_train[target])
        xgb_pred = xgb_model.predict(X_test)
        model_predictions[target]['xgb'] = xgb_pred
        
        # Model 2: Random Forest (from sklearn)
        from sklearn.ensemble import RandomForestRegressor
        rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_model.fit(X_train, y_train[target])
        rf_pred = rf_model.predict(X_test)
        model_predictions[target]['rf'] = rf_pred
        
        # Model 3: LightGBM (if available)
        try:
            import lightgbm as lgb
            lgb_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
            lgb_model.fit(X_train, y_train[target])
            lgb_pred = lgb_model.predict(X_test)
            model_predictions[target]['lgb'] = lgb_pred
        except ImportError:
            print("LightGBM not installed. Skipping this model.")
        
        # Simple average ensemble
        ensemble_preds = np.mean([pred for pred in model_predictions[target].values()], axis=0)
        
        # Evaluate ensemble
        rmse = np.sqrt(mean_squared_error(y_test[target], ensemble_preds))
        mae = mean_absolute_error(y_test[target], ensemble_preds)
        r2 = r2_score(y_test[target], ensemble_preds)
        
        ensemble_results[target] = {
            'rmse': rmse,
            'mae': mae,
            'r2': r2,
            'predictions': ensemble_preds
        }
        
        print(f"Ensemble for {target} - RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")
        
        # Compare with individual models
        print("Individual model performance:")
        for model_name, preds in model_predictions[target].items():
            model_rmse = np.sqrt(mean_squared_error(y_test[target], preds))
            model_r2 = r2_score(y_test[target], preds)
            print(f"  {model_name} - RMSE: {model_rmse:.4f}, R²: {model_r2:.4f}")
    
    return ensemble_results, model_predictions

# 7. Feature Importance Analysis
def analyze_feature_importance(model, features):
    """
    Analyze and visualize feature importance from a trained model
    """
    importance = model.feature_importances_
    
    # Create dataframe for importance
    feature_importance = pd.DataFrame({
        'Feature': features,
        'Importance': importance
    }).sort_values('Importance', ascending=False)
    
    # Plot feature importance
    plt.figure(figsize=(12, 8))
    plt.barh(feature_importance['Feature'][:15], feature_importance['Importance'][:15])
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.title('Top 15 Feature Importance')
    plt.tight_layout()
    plt.show()
    
    return feature_importance

# 8. Explainability with SHAP
def shap_analysis(model, X_test):
    """
    Perform SHAP analysis to explain model predictions
    Note: Requires SHAP package
    """
    try:
        import shap
        
        # Initialize SHAP explainer
        explainer = shap.TreeExplainer(model)
        
        # Calculate SHAP values
        shap_values = explainer.shap_values(X_test)
        
        # Summary plot
        shap.summary_plot(shap_values, X_test)
        
        # Dependence plots for top features
        feature_importance = pd.DataFrame({
            'Feature': X_test.columns,
            'Importance': np.abs(shap_values).mean(0)
        }).sort_values('Importance', ascending=False)
        
        top_features = feature_importance['Feature'][:3].values
        
        for feature in top_features:
            shap.dependence_plot(feature, shap_values, X_test)
            
        return shap_values, explainer
        
    except ImportError:
        print("SHAP not installed. Skip SHAP analysis.")
        return None, None

# 9. Forecast Future Prices
def forecast_future_prices(model, last_data, features, future_days=30):
    """
    Forecast future prices based on the latest data
    """
    # Create future dates
    last_date = pd.to_datetime(last_data[['year', 'month', 'date']].iloc[-1].assign(day=lambda x: x['date']))
    future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=future_days)
    
    # Create dataframe for future data
    future_df = pd.DataFrame({
        'full_date': future_dates,
        'year': future_dates.year,
        'month': future_dates.month,
        'date': future_dates.day
    })
    
    # Add constant features from last known data
    for col in ['district_name_enc', 'commodity_name_enc', 'state_name_enc']:
        future_df[col] = last_data[col].iloc[-1]
    
    # Add time features
    future_df['month_sin'] = np.sin(2 * np.pi * future_df['month']/12)
    future_df['month_cos'] = np.cos(2 * np.pi * future_df['month']/12)
    future_df['day_of_year'] = future_df['full_date'].dt.dayofyear
    future_df['day_of_year_sin'] = np.sin(2 * np.pi * future_df['day_of_year']/365)
    future_df['day_of_year_cos'] = np.cos(2 * np.pi * future_df['day_of_year']/365)
    
    # Add trend
    future_df['trend'] = np.arange(len(last_data), len(last_data) + len(future_df))
    
    # Initialize predictions
    predictions = []
    
    # For each day, predict and update lag features
    current_df = last_data.copy().tail(30)  # Use last 30 days to calculate initial lags
    
    for i in range(future_days):
        # Current day to predict
        current_day = future_df.iloc[i:i+1].copy()
        
        # Calculate lag features based on previous predictions and historical data
        for lag in [1, 3, 7, 14, 30]:
            if lag <= i:
                # Use previous predictions
                pred_idx = i - lag
                current_day['avg_modal_price_lag_{}'.format(lag)] = predictions[pred_idx]['avg_modal_price']
                current_day['avg_min_price_lag_{}'.format(lag)] = predictions[pred_idx]['avg_min_price']
                current_day['avg_max_price_lag_{}'.format(lag)] = predictions[pred_idx]['avg_max_price']
            else:
                # Use historical data
                hist_idx = -lag + i
                if hist_idx < 0 and abs(hist_idx) <= len(current_df):
                    current_day['avg_modal_price_lag_{}'.format(lag)] = current_df['avg_modal_price'].iloc[hist_idx]
                    current_day['avg_min_price_lag_{}'.format(lag)] = current_df['avg_min_price'].iloc[hist_idx]
                    current_day['avg_max_price_lag_{}'.format(lag)] = current_df['avg_max_price'].iloc[hist_idx]
                else:
                    # Fallback if we don't have enough history
                    current_day['avg_modal_price_lag_{}'.format(lag)] = current_df['avg_modal_price'].mean()
                    current_day['avg_min_price_lag_{}'.format(lag)] = current_df['avg_min_price'].mean()
                    current_day['avg_max_price_lag_{}'.format(lag)] = current_df['avg_max_price'].mean()
        
        # Calculate rolling statistics
        for window in [7, 14, 30]:
            # Use combination of historical + predicted values
            combined_hist = list(current_df['avg_modal_price'].tail(min(window, len(current_df))))
            
            # Add predictions made so far if any
            if i > 0:
                pred_values = [p['avg_modal_price'] for p in predictions[-min(i, window-len(combined_hist)):]]
                combined_hist.extend(pred_values)
            
            # Calculate statistics
            if combined_hist:
                current_day['avg_modal_price_rolling_mean_{}'.format(window)] = np.mean(combined_hist[-min(window, len(combined_hist)):])
                current_day['avg_modal_price_rolling_std_{}'.format(window)] = np.std(combined_hist[-min(window, len(combined_hist)):]) if len(combined_hist) > 1 else 0
            else:
                current_day['avg_modal_price_rolling_mean_{}'.format(window)] = current_df['avg_modal_price'].mean()
                current_day['avg_modal_price_rolling_std_{}'.format(window)] = current_df['avg_modal_price'].std()
                
            # Repeat for min price
            combined_hist = list(current_df['avg_min_price'].tail(min(window, len(current_df))))
            if i > 0:
                pred_values = [p['avg_min_price'] for p in predictions[-min(i, window-len(combined_hist)):]]
                combined_hist.extend(pred_values)
            
            if combined_hist:
                current_day['avg_min_price_rolling_mean_{}'.format(window)] = np.mean(combined_hist[-min(window, len(combined_hist)):])
                current_day['avg_min_price_rolling_std_{}'.format(window)] = np.std(combined_hist[-min(window, len(combined_hist)):]) if len(combined_hist) > 1 else 0
            else:
                current_day['avg_min_price_rolling_mean_{}'.format(window)] = current_df['avg_min_price'].mean()
                current_day['avg_min_price_rolling_std_{}'.format(window)] = current_df['avg_min_price'].std()
            
            # Repeat for max price
            combined_hist = list(current_df['avg_max_price'].tail(min(window, len(current_df))))
            if i > 0:
                pred_values = [p['avg_max_price'] for p in predictions[-min(i, window-len(combined_hist)):]]
                combined_hist.extend(pred_values)
            
            if combined_hist:
                current_day['avg_max_price_rolling_mean_{}'.format(window)] = np.mean(combined_hist[-min(window, len(combined_hist)):])
                current_day['avg_max_price_rolling_std_{}'.format(window)] = np.std(combined_hist[-min(window, len(combined_hist)):]) if len(combined_hist) > 1 else 0
            else:
                current_day['avg_max_price_rolling_mean_{}'.format(window)] = current_df['avg_max_price'].mean()
                current_day['avg_max_price_rolling_std_{}'.format(window)] = current_df['avg_max_price'].std()
        
        # Make predictions using the model
        X_day = current_day[features]
        
        # Fill any remaining NaN values
        X_day = X_day.fillna(method='ffill')
        if X_day.isna().any().any():
            X_day = X_day.fillna(0)
        
        # Predict all three targets
        pred = {
            'date': current_day['full_date'].iloc[0],
            'avg_modal_price': model['avg_modal_price'].predict(X_day)[0],
            'avg_min_price': model['avg_min_price'].predict(X_day)[0],
            'avg_max_price': model['avg_max_price'].predict(X_day)[0]
        }
        
        predictions.append(pred)
    
    # Convert predictions to dataframe
    predictions_df = pd.DataFrame(predictions)
    
    return predictions_df

# Main execution
if __name__ == "__main__":
    # Example usage:
    # 1. First, prepare the data with time series features
    processed_df = create_time_features(df)
    
    # 2. Define features for modeling 
    features = [
        # Original features
        'month', 'year', 'date', 'district_name_enc', 'commodity_name_enc', 'state_name_enc',
        # Time features
        'month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos', 'trend',
        # Add lag and rolling statistics as needed
    ]
    
    targets = ['avg_modal_price', 'avg_min_price', 'avg_max_price']
    
    # 3. Train ensemble model
    models, results, feature_importances = train_xgboost_time_series()
    
    # 4. For a specific commodity-location combination, use SARIMA
    sarima_model, forecast, rmse, mae = train_sarima_for_specific_combination(
        commodity_id=1,  # Replace with actual ID
        state_id=1,      # Replace with actual ID
        district_id=1    # Replace with actual ID
    )
    
    # 5. Forecast future prices
    forecast_df = forecast_future_prices(models, processed_df.tail(30), features, future_days=30)
    
    print("Forecasted prices for the next 30 days:")
    print(forecast_df[['date', 'avg_modal_price', 'avg_min_price', 'avg_max_price']].head())